In [1]:
import numpy as np
import pandas as pd
import locale
import json
import re
import pymorphy2
import torch
import accelerate
from datetime import datetime
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig

/Users/alexanderknyshov/Desktop/LLM/Data/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Loading JSON-dataset to dictionary
with open('/Users/alexanderknyshov/Desktop/LLM/Data/datasets/train_set.json', 'r') as json_file:
    data = json.load(json_file)

In [3]:
def format_text(text):
    news_text = "form a recommendation using news: " + " ".join(text['news'])
    recommendation_text = " ".join(text['recommendations'])
    return {'input_text': news_text, 'target_text': recommendation_text}

# Formatting original dataset
dataset = Dataset.from_dict(data)
dataset = dataset.map(format_text)
# Removing unneccessery columns
dataset = dataset.remove_columns(['news', 'recommendations'])

Map: 100%|██████████| 5177/5177 [00:02<00:00, 2003.12 examples/s]


In [4]:
# Splitting into train and validate data
split_ratio = 0.8
split_index = int(len(data['news']) * split_ratio)

train_news = data['news'][:split_index]
train_recommendations = data['recommendations'][:split_index]
train_data = {'news': train_news, 'recommendations': train_recommendations}

val_news = data['news'][split_index:]
val_recommendations = data['recommendations'][split_index:]
val_data = {'news': val_news, 'recommendations': val_recommendations}

# Returning to the format needed
train_dataset = Dataset.from_dict(train_data)
train_dataset = train_dataset.map(format_text)
train_dataset = train_dataset.remove_columns(['news', 'recommendations'])

val_dataset = Dataset.from_dict(val_data)
val_dataset = val_dataset.map(format_text)
val_dataset = val_dataset.remove_columns(['news', 'recommendations'])

Map: 100%|██████████| 1036/1036 [00:00<00:00, 2857.39 examples/s]


In [5]:
# Loading the tokenizer
tokenizer = T5Tokenizer.from_pretrained('sberbank-ai/ruT5-base')

# Method for data tokenization
def tokenize_data(example):
    input_encodings = tokenizer(example['input_text'], padding='max_length', truncation=True, max_length=512)
    target_encodings = tokenizer(example['target_text'], padding='max_length', truncation=True, max_length=256)

    example['input_ids'] = input_encodings['input_ids']
    example['attention_mask'] = input_encodings['attention_mask']
    example['labels'] = target_encodings['input_ids']
    return example

# Train and validation data tokenization
tokenized_train_dataset = train_dataset.map(tokenize_data)
tokenized_val_dataset = val_dataset.map(tokenize_data)

# Getting rid of text columns
tokenized_train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/Users/alexanderknyshov/Desktop/LLM/Data/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 1036/1036 [00:01<00:00, 529.67 examples/s]


In [6]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [7]:
lora_config = LoraConfig(
    r=16,                       # Ранг, может быть изменен в зависимости от задачи
    lora_alpha=32,             # Гиперпараметр, который можно настроить
    lora_dropout=0.1,          # Вероятность дропаута для LoRA
    target_modules=["q", "v"]  # Модули, которые будут адаптированы
)

In [8]:
# Model Loading
model = T5ForConditionalGeneration.from_pretrained('sberbank-ai/ruT5-base').to(device)
model = get_peft_model(model, lora_config)

# Defining training params
training_args = TrainingArguments(
    output_dir= '/Users/alexanderknyshov/Desktop/LLM/Data/model',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=0.1,
    evaluation_strategy="no",
    save_strategy="epoch",   
    logging_dir='/Users/alexanderknyshov/Desktop/LLM/Data/model/logs',
    logging_steps=10,
    save_total_limit=2,
    metric_for_best_model='eval_loss',
)

# Metrics count
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    loss = sum([1 if pred == label else 0 for pred, label in zip(decoded_preds, decoded_labels)]) / len(decoded_preds)
    return {'accuracy': loss}

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/Users/alexanderknyshov/Desktop/LLM/Data/venv/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
trainer.train()
torch.mps.empty_cache()
# Evaluation
#eval_results = trainer.evaluate()

# Results
#print(f"Validation results: {eval_results}")

  5%|▍         | 10/208 [00:05<01:48,  1.82it/s]

{'loss': 13.4203, 'grad_norm': 21.546478271484375, 'learning_rate': 4.7596153846153844e-05, 'epoch': 0.0}


 10%|▉         | 20/208 [00:11<01:42,  1.84it/s]

{'loss': 11.8443, 'grad_norm': 4.979017734527588, 'learning_rate': 4.519230769230769e-05, 'epoch': 0.01}


 14%|█▍        | 30/208 [00:16<01:36,  1.84it/s]

...

{'loss': 7.6538, 'grad_norm': 3.309844970703125, 'learning_rate': 4.326923076923077e-06, 'epoch': 0.09}


 96%|█████████▌| 200/208 [01:49<00:04,  1.83it/s]

{'loss': 7.4033, 'grad_norm': 3.3401286602020264, 'learning_rate': 1.9230769230769234e-06, 'epoch': 0.1}


100%|██████████| 208/208 [01:55<00:00,  1.80it/s]

{'train_runtime': 115.4811, 'train_samples_per_second': 3.586, 'train_steps_per_second': 1.801, 'train_loss': 8.637595066657433, 'epoch': 0.1}


In [10]:
# Замените 'path/to/saved/model' на путь к вашей сгенерированной модели
model = T5ForConditionalGeneration.from_pretrained('/Users/alexanderknyshov/Desktop/LLM/Data/model/checkpoint-208').to(device)
tokenizer = T5Tokenizer.from_pretrained('/Users/alexanderknyshov/Desktop/LLM/Data/model/checkpoint-208')

# Переводим модель в режим оценки
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): lora.Linear(
                (base_layer): Linear(in_features=768, out_features=768, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=768, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k): Linear(in_featu

In [11]:
inp = "form a recommendation using news: " + " ".join(["экономическая", "политическая", "нестабильность", "последнего", "времени", "привела", "падению", "продаж", "одежды", "россии", "сравнению", "аналогичным", "периодом", "прошлого", "года", "отмечают", "эксперты", "мае", "года", "оперируют", "данными", "период", "февраля", "апреля", "такое", "снижение", "спроса", "связывают", "двумя", "факторами", "вопервых", "уходом", "российского", "рынка", "крупных", "западных", "сетей", "массмаркета", "вовторых", "вынужденным", "снижением", "расходов", "многих", "россиянгазпром", "смирился", "новый", "экспортный", "маршрут", "северный", "поток", "который", "построен", "запущен", "эксплуатацию", "сможет", "набрать", "максимальную", "скорость", "поставок", "голубого", "топлива", "европу", "года", "срока", "трубам", "лучшем", "случае", "сможет", "экспортироваться", "лишь", "половина", "заявленных", "объемов", "причем", "лишь", "случае", "соответствующее", "разрешение", "даст", "германияевросоюз", "полон", "решимости", "принять", "шестой", "пакет", "санкций", "котором", "конца", "года", "предусмотрен", "полный", "отказ", "закупок", "российской", "нефти", "нефтепродуктов", "мк", "пообщался", "экспертами", "выяснить", "это", "выльется", "бюджета", "россии", "валютных", "поступленийзападные", "экономические", "издания", "продолжают", "верить", "россию", "мнению", "авторитетного", "британского", "журнала", "the", "economist", "наша", "страна", "несмотря", "связанные", "санкционным", "прессингом", "проблемы", "находит", "способы", "справиться", "предъявляемыми", "вызовами", "например", "поднятая", "рекордного", "уровня", "ключевая", "ставка", "цб", "позволила", "стабилизировать", "курс", "рубля", "вернуть", "российскую", "валюту", "уровень", "октября", "прошлого", "года", "европа", "боится", "остаться", "российских", "ресурсов", "государства", "ес", "начала", "мая", "резко", "нарастили", "запасы", "газа", "частности", "германия", "начала", "заполнять", "крупнейшее", "хранилище", "западной", "европе", "поставки", "финляндию", "выросли", "раза", "причины", "грядущий", "отопительный", "сезон", "желание", "еврокомиссии", "ввести", "эмбарго", "поставки", "россии", "наиболее", "уязвимыми", "случае", "сокращения", "импорта", "окажутся", "австрия", "финляндия", "литвапо", "данным", "газеты", "financial", "time", "западные", "банки", "отложили", "млрд", "убытки", "случае", "ухода", "россии", "причиной", "сворачивания", "прибыльного", "бизнеса", "нашей", "стране", "мировых", "финансовых", "гигантов", "являются", "введенные", "международные", "санкции", "пострадает", "банковская", "система", "россии", "отразиться", "клиентах", "мощный", "хлопок", "дверью", "стороны", "крупнейших", "банковских", "групп", "мк", "выяснил", "экспертов", "банк", "россии", "начал", "публиковать", "наиболее", "популярные", "схемы", "которые", "используют", "мошенники", "кражи", "денег", "счетов", "банковских", "карт", "россиян", "преступники", "стремятся", "оказать", "потенциальную", "жертву", "психологическое", "воздействие", "надавить", "эмоции", "отключить", "критическое", "мышление", "обошли", "вниманием", "последние", "новости", "санкциях", "блокировке", "карт", "эксперты", "рассказали", "поддаться", "манипулированию", "голод", "главная", "угроза", "нынешней", "российской", "турбулентности", "однако", "похоже", "мысль", "году", "неплохо", "посадить", "картошку", "посетила", "очень", "многих", "городских", "жителей", "результате", "подмосковных", "рынках", "образовался", "дефицит", "семенного", "картофеля", "цены", "взлетели", "продают", "картошку", "посев", "тюльпаны", "клубнями", "рублей", "штуку", "привыкшую", "санкциям", "россию", "трудно", "удивить", "какимито", "новыми", "ограничениями", "москва", "находит", "зеркальный", "ассиметричный", "ответ", "неизвестно", "несет", "большие", "финансовые", "потери", "ожиданиях", "отказа", "ес", "закупок", "российского", "газа", "нашлась", "свои", "контрмеры", "объемы", "которые", "сегодня", "поставляем", "страны", "европы", "могут", "перенаправлены", "азиатскотихоокеанский", "регион", "атр", "сбербанк", "оперативно", "отреагировал", "решение", "правительства", "рф", "увеличить", "максимальную", "сумму", "кредита", "которую", "брать", "покупку", "квартиры", "использованием", "льготной", "ставки", "мая", "покупатели", "недвижимости", "москве", "московской", "области", "санктпетербурге", "ленинградской", "области", "могут", "рассчитывать", "кредиты", "млн", "рублейстраны", "европы", "раскололись", "линии", "российского", "газа", "одни", "согласны", "схему", "оплаты", "конвертацией", "рубли", "другие", "отказались", "напрочь", "газ", "россии", "ло", "сих", "пор", "закрывал", "трети", "потребностей", "жителей", "континента", "энергоносителях", "европейцам", "приходится", "искать", "альтернативных", "поставщиков", "топлива", "россии", "нужно", "обеспечить", "аналогичный", "рынок", "сбыта", "сможет", "наша", "страна", "повернуть", "свой", "энергетический", "поток", "запада", "восток", "объяснили", "экономистыверховный", "представитель", "европейского", "союза", "иностранным", "делам", "политике", "безопасности", "жозеп", "боррель", "предложил", "странамчленам", "объединения", "обсудить", "возможность", "изъятия", "замороженных", "российских", "валютных", "резервов", "поддержки", "украины", "мид", "рф", "такую", "инициативу", "назвали", "полным", "беззаконием", "разрушением", "основы", "международных", "отношений", "эксперт", "оценил", "пойдет", "запад", "чрезвычайный", "шаг", "ответить", "россия", "большие", "магазины", "компании", "закрываются", "уходят", "россии", "фоне", "санкций", "нередко", "создавая", "колоссальный", "общественный", "резонанс", "тени", "остаётся", "малый", "бизнес", "мк", "собрал", "истории", "нескольких", "предпринимателей", "которые", "сегодня", "остались", "своими", "проблемами", "опустили", "руки", "специальная", "военная", "операция", "развязанная", "западом", "санкционная", "политика", "против", "россии", "оформили", "завершение", "политического", "хх", "века", "мировой", "арене", "снижается", "часто", "сходит", "роль", "традиционных", "мировых", "политикоэкономических", "игроковнынешний", "год", "стать", "самым", "провальным", "отрезком", "времени", "экономики", "нашей", "страны", "последние", "десятка", "лет", "данным", "агентства", "bloomberg", "недрах", "минфина", "рф", "прорабатывается", "прогноз", "соответствии", "которым", "потери", "отечественного", "ввп", "м", "составят", "наблюдалось", "годаминэкономразвития", "порадовало", "россиян", "новыми", "статистическими", "выкладками", "инфляция", "стране", "разогналась", "годовом", "выражении", "это", "вдвое", "прошлогоднего", "индекса", "роста", "потребительских", "цен", "несколько", "выше", "целевых", "показателей", "достичь", "которых", "обещало", "населению", "правительствофонды", "обязательного", "медицинского", "пенсионного", "социального", "страхования", "получат", "правительства", "трлн", "рублей", "качестве", "компенсации", "отсрочку", "уплаты", "страховых", "взносов", "предоставленную", "государством", "ряду", "отечественных", "предприятий", "снижения", "финансовой", "нагрузки", "словам", "премьерминистра", "михаила", "мишустина", "денежный", "транш", "позволит", "социальным", "фондам", "продолжить", "бесперебойную", "выдачу", "людям", "пенсий", "пособий", "также", "послужит", "залогом", "оказания", "бесплатной", "медицинской", "помощиидея", "овладевшая", "массами", "становится", "материальной", "силой", "учили", "классики", "марксизма", "точки", "зрения", "идея", "конфискации", "российских", "валютных", "резервов", "весьма", "близка", "воплощению", "стремительно", "покоряет", "евроатлантический", "политбомонд", "свидетельствуют", "частности", "недавние", "высказывания", "главы", "евродипломатии", "жозепа", "борреля", "россия", "спасти", "свои", "арестованные", "активы", "экспроприации"])
input_ids = tokenizer.encode(inp, return_tensors='pt').to(device)  

In [15]:
with torch.no_grad():  # Выключаем градиенты для оценки
    outputs = model.generate(
    input_ids,               # Входные данные
    max_length=50,           # Максимальная длина генерации
    do_sample=True,          # Включаем семплирование для большего разнообразия
    temperature=1.2,         # Увеличиваем случайность при генерации
    top_k=50,                # Ограничиваем выбор до 50 наиболее вероятных слов
    top_p=0.9,               # Вероятностное отсечение
    no_repeat_ngram_size=2,  # Предотвращаем повторение одинаковых биграмм
    length_penalty=1.0       # Регулируем длину генерации
) # Установите max_length по мере необходимости

# Декодируем сгенерированные идентификаторы в текст
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
generated_text

'<pad>ов на отечествеся рынк незакупленных товарных позиций и товаре по-.gdd-контрактопродуктов цен с ом в ах ов. грива sijо'